## Model

In [2]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━

In [10]:
import flair, torch
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter
from flair.data import Sentence

from collections import defaultdict

In [4]:
torch.cuda.is_available()

True

In [44]:
!nvidia-smi

Fri Feb 17 13:41:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    33W /  70W |   9310MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
flair.device = torch.device("cuda:0")

In [7]:
splitter = SegtokSentenceSplitter()
#tagger = SequenceTagger.load('ner')
tagger = SequenceTagger.load('ner-fast')

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:629: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2023-02-17 11:23:24,048 loading file /root/.flair/models/ner-english-fast/4c58e7191ff952c030b82db25b3694b58800b0e722ff15427f527e1631ed6142.e13c7c4664ffe2bbfa8f1f5375bd0dced866b8c1dd7ff89a6d705518abf0a611
2023-02-17 11:23:26,060 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [9]:
def ner_eval(stats: dict, new: dict):
    splitted = splitter.split(new["content"])
    splitted.append(Sentence(new["title"]))
    if isinstance(new["description"], str):
        splitted.extend(
            splitter.split(new["description"])
        )

    tagger.predict(splitted)

    buf_stats = {
        "PER":  defaultdict(int),
        "LOC":  defaultdict(int),
        "ORG":  defaultdict(int),
        "MISC": defaultdict(int)
    }     # костыль

    # число вхождений в текст
    for sentence in splitted:
        for entity in sentence.get_spans("ner"):
            #print(entity)
            buf_stats[entity.tag][entity.text] += 1 

    for key in buf_stats.keys():
        for entity in buf_stats[key].keys():
            stats[key][entity].append((new["index"], buf_stats[key][entity]))

## Data

In [12]:
from pprint import pprint
import json
from google.colab import drive

In [15]:
drive.mount('/content/up_nlp/', force_remount=True)

Mounted at /content/up_nlp/


In [16]:
with open("/content/up_nlp/MyDrive/up_nlp/data.json", "r") as data_file:
    data = json.load(data_file)

len(data)

42635

In [18]:
with open("/content/up_nlp/MyDrive/up_nlp/stats.json", "r") as stats_file:
    stats = json.load(stats_file)

for key in ["PER", "LOC", "ORG", "MISC"]:
    stats[key] = defaultdict(list, stats[key])

{key: len(stats[key]) for key in ["PER", "LOC", "ORG", "MISC"]}

{'PER': 27780, 'LOC': 7065, 'ORG': 18536, 'MISC': 11428}

## Evaluating

In [28]:
from tqdm import tqdm

In [58]:
N_SAMPLES = 2635

In [59]:
len(data)

42635

In [60]:
stats["last_new_indx"]

39999

In [61]:
ner_samples_failed = []
for i in tqdm(range(stats["last_new_indx"]+1, stats["last_new_indx"]+N_SAMPLES+1)):
    try:
        ner_eval(stats, data[i])
    except:
        ner_samples_failed.append(data[i]["index"])

    if (i+1) % 1000 == 0:
        stats["last_new_indx"] += 1000
        with open("/content/up_nlp/MyDrive/up_nlp/stats.json", "w") as stats_file:
            stats_file.write(json.dumps(stats))
            print(f"Dumped after {(stats['last_new_indx'])}'th sample")
print(ner_samples_failed)

 38%|███▊      | 1000/2635 [07:10<30:27,  1.12s/it]

Dumped after 40999'th sample


 76%|███████▌  | 2000/2635 [13:44<12:55,  1.22s/it]

Dumped after 41999'th sample


100%|██████████| 2635/2635 [18:08<00:00,  2.42it/s]

[]


In [62]:
{key: len(stats[key]) for key in ["PER", "LOC", "ORG", "MISC"]}

{'PER': 185089, 'LOC': 52875, 'ORG': 108032, 'MISC': 88084}

In [64]:
del stats['last_new_indx']

In [65]:
with open("/content/up_nlp/MyDrive/up_nlp/stats.json", "w") as stats_file:
    stats_file.write(json.dumps(stats))

In [66]:
stats["last_new_indx"]

KeyError: ignored